In [2]:
import pandas as pd

In [5]:
path_xlsx_avance = r'C:/Documents/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/AVANCE DE COSECHA - PRUEBA - Copy.xlsx'

In [6]:
xlsx_resumen = pd.read_excel(path_xlsx_avance, sheet_name='AVANCE_FECHAS')

In [7]:
# elimina el ultimo registro, para descartar los totales
xlsx_resumen = xlsx_resumen.drop(xlsx_resumen.index[-1])

In [8]:
xlsx_resumen

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGADO
0,1,2024-05-16,4.127295,221.918352,NaN
1,1,2024-05-29,NaN,NaN,1906.18
2,1,2024-05-31,45.972261,2338.489409,970.59
3,1,2024-06-05,63.240335,2836.57669,2483.7
4,1,2024-06-10,48.153913,2486.75969,2421.99
...,...,...,...,...,...
3252,NaN,NaT,NaN,NaN,NaN
3253,NaN,NaT,NaN,NaN,NaN
3254,NaN,NaT,NaN,NaN,NaN
3255,NaN,NaT,NaN,NaN,NaN


In [47]:
def colapsar_tabla(df_param):
    suma_acumulador = [None] * len(df_param)
    acumulador = 0
    contador = 0
    for i, r in df_param.iterrows():
        if(r['AREA'] == 0):
            acumulador += r['ENTREGADO']
            contador += 1
        else:
            acumulador += r['ENTREGADO']
            suma_acumulador[contador] = acumulador
            acumulador = 0
            contador += 1
            continue
    if suma_acumulador[-1] == None:
        suma_acumulador[-1] = acumulador
        
    print(df_param)
    print(suma_acumulador)
    df_param['ENTREGAS'] = suma_acumulador
    df_param = df_param.drop(columns=['ENTREGADO'])
    
    df_param = df_param[df_param['AREA']!=0]
    return df_param

In [48]:
suma_acumulador = [None] * len(grupo)

In [34]:
type(suma_acumulador[0])

NoneType

In [49]:
list_cod_cos = list(set(xlsx_resumen['COD_COS']))

In [50]:
list_df = []

In [51]:
grupo = xlsx_resumen[xlsx_resumen['COD_COS']==17].fillna(0)

In [52]:
grupo

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGADO
112,17,2024-06-18,22.298621,1288.302440,2358.23
113,17,2024-06-20,12.108021,908.951624,762.37
114,17,2024-07-17,21.077437,1619.120484,2054.73
115,17,2024-07-25,0.000000,0.000000,81.57
116,17,2024-08-10,0.000000,0.000000,1252.70


In [53]:
result = colapsar_tabla(grupo)

     COD_COS      FECHA       AREA     ESTIMADO  ENTREGADO
112       17 2024-06-18  22.298621  1288.302440    2358.23
113       17 2024-06-20  12.108021   908.951624     762.37
114       17 2024-07-17  21.077437  1619.120484    2054.73
115       17 2024-07-25   0.000000     0.000000      81.57
116       17 2024-08-10   0.000000     0.000000    1252.70
[2358.23, 762.37, 2054.73, None, 1334.27]


In [54]:
result

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGAS
112,17,2024-06-18,22.298621,1288.302440,2358.23
113,17,2024-06-20,12.108021,908.951624,762.37
114,17,2024-07-17,21.077437,1619.120484,2054.73


In [9]:
for i in list_cod_cos:
    grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
    result = colapsar_tabla(grupo)
    list_df.append(result)

In [10]:
df_combinado = pd.concat(list_df, ignore_index=True)

C:\Users\bismarksr\AppData\Local\Temp\ipykernel_11204\69313519.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combinado = pd.concat(list_df, ignore_index=True)


In [11]:
df_combinado

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGAS
0,1,2024-05-16 00:00:00,4.127295,221.918352,0.00
1,1,2024-05-31 00:00:00,45.972261,2338.489409,2876.77
2,1,2024-06-05 00:00:00,63.240335,2836.576690,2483.70
3,1,2024-06-10 00:00:00,48.153913,2486.759690,2421.99
4,1,2024-06-18 00:00:00,20.112090,815.611812,2230.23
...,...,...,...,...,...
2204,576,2024-07-17 00:00:00,7.923825,428.124620,463.90
2205,576,2024-07-25 00:00:00,5.398911,250.407765,0.00
2206,577,2024-07-20 00:00:00,1.435498,0.000000,89.57
2207,578,2024-07-17 00:00:00,7.535220,0.000000,285.20


In [12]:
df_combinado.to_excel('C:/Documents/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')